In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib notebook
matplotlib.rcParams['figure.figsize'] = (9.0, 6.0)

In [2]:
import numpy as np

def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

In [3]:
import redis

r = redis.StrictRedis(host='172.17.0.1', port=6379, db=0)

ModuleNotFoundError: No module named 'redis'

In [ ]:
import pandas as pd

def get_data_from_pattern(pattern, pref='OUT'):
    list_of_keys = [key for key in r.keys(pref + '*' + pattern + '*') if 'time' not in str(key)]
    list_of_time = [key for key in r.keys(pref + '*' + pattern + '*time')]
    
    res = {}
    
    for (key_v, key_t) in zip(list_of_keys, list_of_time):
        value = list(map(float, r.lrange(key_v, 0, -1)))
        index = list(map(float, r.lrange(key_t, 0, -1)))

        index = pd.to_datetime(index, unit='s')

        res[key_v.decode("utf-8").replace(pref + '_', '')] = pd.Series(value, index=index)

    return res

In [ ]:
results_th = get_data_from_pattern('*t_sup*')

In [ ]:
for key, value in results_th.items():
    rol = value.rolling(60, center=True).mean()
    
    freq = '3600S'
    sam = value.resample(freq).mean()
    
    plt.plot(value.index, value, '+-', alpha=0.5, label=key,  markersize=2.5)
    plt.plot(rol.index, rol, alpha=0.5, label=key+'_rolling_average')
    plt.plot(sam.index, sam, '--s', label=key+'_resample_'+freq, markersize=2.5)
    
    
    # rolling.plot(label=key+'_rolling_average', linestyle=None, marker='.', markersize=1)
    

plt.legend()
plt.show()